In [1]:
import sys

sys.path.append("../../src")

import pickle
from tuneable_counterfactuals_explainer.explainer import Explainer

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
model = pickle.load(open("model/model.pkl", "rb"))
training_data = pickle.load(open("model/train_X.pkl", "rb"))

In [4]:
explainer = Explainer(
    model,
    training_data,
    "target",
    bounding_method="quantile",
    number_samples=20,
    changeability_scores={
        # 'bmi': 0.5,
        "weight_first": 0.5,
        "platelet_first": 1,
        # 'sodium_first': 1,
        # 'potassium_first':1,
        "tco2_first": 1,
        "chloride_first": 1,
        "creatinine_first": 1,
    },
)

In [5]:
res = explainer.explain(
    training_data.iloc[0],
    store_historical_explainers=True,
    # additional_threshold=0.45,
)

206it [00:02, 82.26it/s]                     

All nodes in this graph have been searched


In [6]:
import graphviz

In [8]:
converter = {
    "weight_first": "Weight",
    "platelet_first": "Platelet",
    "tco2_first": "TCO2",
    "chloride_first": "Chloride",
    "creatinine_first": "Creatinine",
}
temp = [
    ((str(i), ", ".join([converter[z] for z in x])), x, y)
    for i, (x, y) in enumerate(res.items())
]

In [9]:
counts = [1, 5, 4, 3, 2, 1, 1, 2, 1, 1, 2, 1, 1]
count_expanded = []
for i in range(len(counts)):
    count_expanded += [i] * counts[i]

In [10]:
temp_2 = [(x, y) for x, y in zip(count_expanded, temp)]

In [12]:
dot = graphviz.Digraph(comment="Search Visualization")
max = 20

for stage in range(1, 10):
    if stage > 1:
        dot.node(
            f"cluster_{stage}",
            label=f"Search Stage {stage-1}",
            style="filled",
            color="lightgrey",
            shape="box",
        )
    else:
        dot.node(
            f"cluster_{stage}",
            label=f"Search Stage {stage-1}",
            color="lightgrey",
            shape="box",
            style="invis",
        )

    if stage > 1:
        dot.body.append(f'"cluster_{stage-1}" -> "cluster_{stage}" [style=invis]\n')

for stage, ((i, x), y, z) in temp_2[:max]:
    if stage == 0:
        dot.node(str(y), "Initial Value")
    else:
        dot.node(str(y), x + "\nScore: " + format(z, "f"))
    if len(y) > 0:
        dot.edge(str(y[:-1]), str(y))
    for sub_stage, ((sub_i, sub_x), sub_y, sub_z) in temp_2[:max]:
        if stage < sub_stage:
            dot.body.append(f'"{str(y)}" -> "{str(sub_y)}" [style=invis]\n')
            break

dot.attr(layout="dot")
# dot.attr(rankdir='LR')
# dot.attr(ranksep='30')
dot.render("doctest-output/round-table.gv").replace("\\", "/")
"doctest-output/round-table.gv.pdf"

'doctest-output/round-table.gv.pdf'